<p>
<font size='5' face='Georgia, Arial'>IIC2115 - Programación como herramienta para la ingeniería</font><br>
</p>

# Acceso a bases de datos con Python

A pesar de que existe una gran variedad de sistemas administradores de bases de datos (SySQL, SQL Server, PostreSQL), estos son en general programas pesados, con muchas funcionalidades, principalmente pensadas en aplicaciones grandes y no con fines académicos.

Para nuestro caso, dado que no necesitamos tanto, utilizaremos SQLite, un administrador altamente compacto y funcional, compatible con SQL, y que está disponible para Windows, Linux y Mac.

## Conexión y consultas

SQLite puede ser accedido de manera directa y transparente a través de Pythom utilizando el modulo **sqlite3**. El siguiente ejemplo muestra como crear una nueva base de datos y ejecutar una serie de consultas en SQL sobre ella.

In [ ]:
import sqlite3  # importamos el módulo de sqlite

# Creamos o abrimos (si ya existe) la base de datos example. 
# En SQLite, las bases de datos, independiente de la cantidad de
# tablas que tengan, son almacenadas en un archivo.
connection = sqlite3.connect('example.db') 
            
# Generamos un cursor al contenido de la base de datos, lo que nos permite 
# acceder al contenido de las tablas y a realizar consultas sobre ellas.   
cursor = connection.cursor()                    

# El método execute permite introducir directamente comandos en SQL. 
# En las siguientes tres llamadas al método, creamos una tabla y luego la poblamos con dos filas    
cursor.execute("CREATE TABLE countries(name TEXT, continent TEXT, population INTEGER, language TEXT)")
cursor.execute("INSERT INTO countries VALUES ('Andorra','Europe',77281, 'Catalan')")
cursor.execute("INSERT INTO countries VALUES ('China','Asia', 1403500365, 'Mandarin')")

# En general, en las bases de datos, los cambios no son incorporados de manera inmediata, 
# con el fin de asegurar la consistencia al acceso de todos los usuarios. Por este motivo, 
# es necesario indicar de manera explicita, usando el método commit, el momento en que 
# se desea agregar o actualizar datos.
connection.commit() 

Podemos verificar la creación de la tabla utilizando el siguiente comando:

In [ ]:
cursor.execute('PRAGMA table_info([countries])')
print(cursor.fetchall())

# Una vez que hemos realizado todas las consultas requeridas, debemos cerrar la conexión con la base de datos.
# Es importante recordar que los cambios que no se hacen efectivos (no se hace commit)
connection.close()

Como se puede ver, el comando anterior nos entrega información relevante, como tipo de dato y posición, para cada una de las columnas de la tabla. Es importante notar que en este momento, no existe una conexión activa con la base de datos, ya que esta fue cerrada con el comando **connection.close()**.

In [ ]:
# este llamado a execute falla, debido a que no existe una conexión abierta con la base de datos.
cursor.execute('PRAGMA table_info([stocks])')
print(cursor.fetchall())

De esta manera, antes de realizar una consulta, es necesario abrir nuevamente una conexión, la que a diferencia del caso anterior, no creará un nuevo archivo *example.db*, ya que este fue creado anteriormente. Es importante notar que no es necesario cerrar siempre la conexión con la base de datos, sólo cuando no se espera interactuar con ella en el plazo inmediato.

Volviendo al ejemplo anterior, si queremos verificar que la inserción fue realizada correctamente, debemos ejecutar una consulta sobre la misma base de datos.

In [ ]:
connection = sqlite3.connect('example.db')
cursor = connection.cursor()
cursor.execute('SELECT * FROM countries')

#obtenemos sólo una fila, si queremos todas, debemos usar fetchall (no cambia el resultado en este caso)
print(cursor.fetchone())
connection.close()

Otra manera de revisar los contenidos de la base de datos, es utilizando una herramienta especializada para esto. Dentro de la gran variedad existente, una opción simple y liviana es [SQLiteStudio](sqlitestudio.pl), que también es multiplataforma (Windows, Linux, Mac).

## Parametrización de consultas

La manera más directa y sencilla de introducir información variable en una consulta, es utilizando *strings*. Por ejemplo, si permitimos a un usuario indicar el nombre del país sobre el cuál se buscará información, es posible construir una consulta parametrizada de la siguiente manera:

In [ ]:
connection = sqlite3.connect('example.db')
cursor = connection.cursor()
country = "'China'"
cursor.execute("SELECT * FROM countries WHERE name = %s" % country)
print(cursor.fetchone())

A pesar de que la consulta anterior entregó el resultado esperado, el usar este tipo de estrategia para parametrizar consultas es un error muy común en la gran mayoría de los programas que interactuan con bases de datos. Esto puede generar grandes problemas de seguridad, debido a un ataque conocido como *SQL Injection*, que consiste en introducir, como dato para una consulta, consultas completas de SQL que pueden causar grandes pérdidas en la base de datos. Veamos el siguiente caso como ejemplo, donde eliminaremos completamente la tabla *countries* utilizando esta técnica:

In [ ]:
connection = sqlite3.connect('example.db')
cursor = connection.cursor()
country = "'China'; DROP TABLE countries;"
# a diferencia de execute, el método executescript permite ejecutar más de una consulta de 
# manera secuencial (esto abre la puerta a la ejecución de código indeseado).
cursor.executescript("SELECT * FROM countries WHERE name = %s" % country)

#verificamos la existencia de la tabla
cursor.execute('PRAGMA table_info([countries])')
print(cursor.fetchall())

Para evitar situaciones de este tipo, es fundamental utilizar una técnica segura, como la **sustitución de parámetros**. En el caso de SQLite en Python, se debe poner el signo **?** en una consulta, cada vez que se desee sustituir posteriormente con un valor. Posteriormente, basta con introducir como una tupla los valores deseados, como segundo parámetro del método *execute*.

In [ ]:
cursor.execute("CREATE TABLE countries(name TEXT, continent TEXT, population INTEGER, language TEXT)")
cursor.execute("INSERT INTO countries VALUES ('Andorra','Europe',77281, 'Catalan')")
cursor.execute("INSERT INTO countries VALUES ('China','Asia', 1403500365, 'Mandarin')")

country = ('Andorra',)
cursor.execute('SELECT * FROM countries WHERE name = ?', country)
print(cursor.fetchall())

A diferencia del uso de *strings*, la sustitución de parámetros, al hacer explícita la existencia de información externa, permite realizar verificaciones de seguridad que impiden la ejecución de código malicioso. De esta manera, si intentamos realizar nuevamente un ataque de *SQL Injection*, el resultado será distinto:

In [ ]:
country = ("'China'; DROP TABLE countries;",)
cursor.execute("SELECT * FROM countries WHERE name = ?", country)
cursor.execute('PRAGMA table_info([countries])')
print(cursor.fetchall())

Muchas veces, deberemos insertar más de un valor simultaneamente en la base de datos. Con el fin de evitar el llamado repetido al método **execute** (o el uso del inseguro método **executescript**, la interfaz de SQLite para Python provee el método **executemany**, que permite ejecutar múltiples consultas de manera simultánea, seteando parámetros individualmente para cada una de ellas:

In [ ]:
more_countries = [('Jamaica', 'America', 2881355, 'English'),
                  ('Chile', 'America', 18006407, 'Spanish'),
                  ('Australia', 'Oceania', 24680100, 'English'),
                 ]
cursor.executemany('INSERT INTO countries VALUES (?,?,?,?)', more_countries)
cursor.execute('SELECT * FROM countries')
cursor.fetchall()

Finalmente, con el fin de hacer más limpia (y formateable) la impresión de los resultados obtenidos por una consulta, es posible utilizar el cursor resultante de una consulta como un iterador:

In [ ]:
for country in cursor.execute('SELECT * FROM countries ORDER BY name'):
    print(country)
connection.close()